In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Scrape All Books Basic info
url = 'https://books.toscrape.com/'
browser.visit(url)
titles = []
prices = []
ratings = []
stocks = []
for x in range(1, 50):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    books = soup.find_all('article', class_='product_pod')
    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").get_text().strip()
        rating = book.p["class"][1]
        stock = book.find("p", class_="instock availability").get_text().strip()
        titles.append(title)
        prices.append(price)
        ratings.append(rating)
        stocks.append(stock)
    if browser.links.find_by_partial_text('next'):
        browser.links.find_by_partial_text('next').click()
    else:
        print("-Done")

In [4]:
# Crate Basic Scrape Data Frame
scrape_df = pd.DataFrame(
                        {'Titles':  titles,
                        'Prices':   prices,
                        'Ratings':  ratings,
                        'Stocks':   stocks})
scrape_df

,Titles,Prices,Ratings,Stocks
0,A Light in the Attic,£51.77,Three,In stock
1,Tipping the Velvet,£53.74,One,In stock
2,Soumission,£50.10,One,In stock
3,Sharp Objects,£47.82,Four,In stock
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock
...,...,...,...,...
975,Icing (Aces Hockey #2),£40.44,Four,In stock
976,"Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)",£45.24,Three,In stock
977,Having the Barbarian's Baby (Ice Planet Barbar...,£34.96,Four,In stock
978,"Giant Days, Vol. 1 (Giant Days #1-4)",£56.76,Four,In stock


In [5]:
#List of all the catergories, was easy to manually do. But I should come back and do it programatically later
temListOfCategories = ["Travel","Mystery","Historical_Fiction","Sequential_Art","Classics","Philosophy","Romance","Womens_Fiction","Fiction","Childrens","Religion","Nonfiction","Music","Default","Science_Fiction","Sports_and_Games","Add_a_comment","Fantasy","New_Adult","Young_Adult","Science","Poetry","Paranormal","Art","Psychology","Autobiography","Parenting","Adult_Fiction","Humor","Horror","History","Food_and_Drink","Christian_Fiction","Business","Biography","Thriller","Contemporary","Spirituality","Academic","Self_Help","Historical","Christian","Suspense","Short_Stories","Novels","Health","Politics","Cultural","Erotica","Crime"]

In [6]:
# Collect Category links
url = 'https://books.toscrape.com/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

Section = soup.find('ul', class_='nav nav-list')
linksSection = Section.find_all('a')
linksSectionStr = []
for x in linksSection:
    linksSectionStr.append(str(x))
categoriesLinks = []
for x in range(len(linksSectionStr)):
    linksSectionStr[x] = re.sub(' +', '', linksSectionStr[x])
    linksSectionStr[x] = re.sub('\n', '', linksSectionStr[x])
    linksSectionStr[x] = re.findall(r'"([^"]*)"', linksSectionStr[x])
    categoriesLinks.append(linksSectionStr[x][0])
#Pop the first one since it was "Books" category which contained all books and wasnt needed
categoriesLinks.pop(0)

'catalogue/category/books_1/index.html'

In [7]:
#Collect the titles within each category and move onto the next
url = 'https://books.toscrape.com/'

generaCollection = []
genraTitles = []
for x in range(len(temListOfCategories)):
    browser.visit(url + categoriesLinks[x])
    title1 = []
    page = True
    while page == True:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        books = soup.find_all('article', class_='product_pod')
        for book in books:
            title = book.h3.a["title"]
            title1.append(title)
        if browser.links.find_by_partial_text('next'):
            browser.links.find_by_partial_text('next').click()
        else:
            print("-Done")
            page = False
    generaCollection.append(title1)

-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done
-Done


In [8]:
# Adds 
toAddToDf = pd.DataFrame()
genreMatch = pd.DataFrame()
counter = True
for x in range(len(temListOfCategories)):
    while counter == True:
        genreMatch = genreMatch.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
        genreMatch['Genre'] = x
        counter = False
    counter = True
    toAddToDf = toAddToDf.append(genreMatch)
    genreMatch.drop(genreMatch.index, inplace=True)
toAddToDf


C:\Users\pereg\AppData\Local\Temp\ipykernel_12024\1162462277.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genreMatch = genreMatch.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
C:\Users\pereg\AppData\Local\Temp\ipykernel_12024\1162462277.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toAddToDf = toAddToDf.append(genreMatch)
C:\Users\pereg\AppData\Local\Temp\ipykernel_12024\1162462277.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genreMatch = genreMatch.append(scrape_df[scrape_df['Titles'].isin(generaCollection[x])])
C:\Users\pereg\AppData\Local\Temp\ipykernel_12024\1162462277.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

,Titles,Prices,Ratings,Stocks,Genre
19,It's Only the Himalayas,£45.17,Two,In stock,0
189,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,Four,In stock,0
268,See America: A Celebration of Our National Par...,£48.87,Three,In stock,0
448,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,Two,In stock,0
496,Under the Tuscan Sun,£37.33,Three,In stock,0
...,...,...,...,...,...
219,Why the Right Went Wrong: Conservatism--From G...,£52.65,Four,In stock,46
383,Equal Is Unfair: America's Misguided Fight Aga...,£56.86,One,In stock,46
212,Amid the Chaos,£36.58,One,In stock,47
200,Dark Notes,£19.19,Five,In stock,48


In [9]:
browser.quit()

In [10]:
toAddToDf = toAddToDf.drop(["Prices","Ratings","Stocks"], axis=1)
genreDf = scrape_df.merge(toAddToDf, how='outer',on='Titles')


In [11]:
#rename genre numbers matching the index of the list
for x in range(len(temListOfCategories)):
    genreDf.loc[genreDf["Genre"] == x, "Genre"] = temListOfCategories[x]


In [12]:
genreDf

,Titles,Prices,Ratings,Stocks,Genre
0,A Light in the Attic,£51.77,Three,In stock,Poetry
1,Tipping the Velvet,£53.74,One,In stock,Historical_Fiction
2,Soumission,£50.10,One,In stock,Fiction
3,Sharp Objects,£47.82,Four,In stock,Mystery
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock,History
...,...,...,...,...,...
977,Icing (Aces Hockey #2),£40.44,Four,In stock,Sports_and_Games
978,"Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)",£45.24,Three,In stock,Sequential_Art
979,Having the Barbarian's Baby (Ice Planet Barbar...,£34.96,Four,In stock,Science_Fiction
980,"Giant Days, Vol. 1 (Giant Days #1-4)",£56.76,Four,In stock,Sequential_Art


In [13]:
#find duplicates
ids = genreDf["Titles"]
genreDf[ids.isin(ids[ids.duplicated()])].sort_values("Titles")

,Titles,Prices,Ratings,Stocks,Genre
236,The Star-Touched Queen,£46.02,Five,In stock,Fantasy
237,The Star-Touched Queen,£46.02,Five,In stock,Fantasy
238,The Star-Touched Queen,£32.30,Five,In stock,Fantasy
239,The Star-Touched Queen,£32.30,Five,In stock,Fantasy


In [14]:
#remove weird duplicates and keep last, even though its the same book they are differnt entries with different prices
genreDf = genreDf.drop_duplicates(subset=['Titles', 'Prices'], keep='last')

In [15]:
#check the length to see if it matches original scrape and it does!
len(genreDf)

980

In [16]:
genreDf.to_csv("Resources/scrapedBooks.csv",index=False)